In [293]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error,mean_squared_error
import xarray as xr
from sklearn.model_selection import train_test_split
import datetime
from sklearn.metrics import accuracy_score
import numpy as np
import math
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import GridSearchCV
import scipy
import time
from keras.wrappers.scikit_learn import KerasRegressor

In [351]:

df=pd.read_csv('KANPUR_DATA_new.csv')
df=df.drop(['SSA_440nm'], axis=1)
df

,Date,AOD_440nm,AE,windspeed,wind_direction,TP,Temp,PBLH,RH,AAI
0,01-01-2001,NaN,NaN,1.055931,286.184844,9.458279e-04,288.39835,310.09170,82.068820,-0.008343
1,02-01-2001,NaN,NaN,2.218344,155.146463,2.809949e-04,287.43002,457.48444,81.131770,NaN
2,03-01-2001,NaN,NaN,2.473740,164.261302,0.000000e+00,286.10706,493.10452,75.176740,NaN
3,04-01-2001,NaN,NaN,2.940446,166.552058,0.000000e+00,285.62973,579.78830,74.007965,0.212354
4,05-01-2001,NaN,NaN,2.979542,184.559102,0.000000e+00,285.24817,504.81683,69.652590,NaN
...,...,...,...,...,...,...,...,...,...,...
7969,27-10-2022,NaN,1.799,1.483994,216.201148,-2.235174e-08,297.78590,419.45193,70.678520,NaN
7970,28-10-2022,NaN,1.800,1.511542,310.986664,-2.235174e-08,297.65674,466.13077,70.421940,NaN
7971,29-10-2022,NaN,1.800,1.569485,356.651682,-2.235174e-08,297.31622,506.78067,70.670320,NaN
7972,30-10-2022,NaN,1.784,1.138774,111.006960,-2.235174e-08,296.92840,522.38770,68.830154,NaN


In [352]:
df= df.replace(-999, np.nan)



df= df.dropna(subset=df.columns.values)




df[["day", "month", "year"]] = df["Date"].str.split('-', n = 3, expand=True)



df=df.drop('Date', axis=1)



df['year']=df['year'].astype('float64')
df['month']=df['month'].astype('float64')
df['day']=df['day'].astype('float64')
df
#df['AOD_440nm']=np.log10(df['AOD_440nm'])

,AOD_440nm,AE,windspeed,wind_direction,TP,Temp,PBLH,RH,AAI,day,month,year
626,0.468706,0.960,1.378377,105.671068,1.768591e-04,302.06506,537.62410,74.383620,0.686675,19.0,9.0,2002.0
633,0.465906,1.800,1.907279,166.984458,5.938299e-05,300.52576,555.50290,78.481834,0.602194,26.0,9.0,2002.0
636,0.469908,1.619,1.800257,189.353322,4.302710e-07,301.04758,492.15730,74.346910,0.220123,29.0,9.0,2002.0
639,0.318449,1.787,2.023659,164.650015,0.000000e+00,300.10574,437.02628,67.287895,-0.334267,2.0,10.0,2002.0
651,0.485983,1.800,2.013939,16.996497,2.147285e-04,299.66187,591.35986,73.569340,0.018496,14.0,10.0,2002.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7689,1.483782,1.799,1.589938,318.283679,5.118977e-04,284.95978,340.98602,84.849740,0.055384,20.0,1.0,2022.0
7690,1.080180,1.799,1.645408,16.194423,-2.235174e-08,286.44742,370.49844,80.635850,0.698075,21.0,1.0,2022.0
7694,0.457155,1.770,2.093941,148.827939,8.586794e-07,286.08680,553.41780,74.524090,-0.238326,25.0,1.0,2022.0
7697,0.258125,1.736,3.354997,174.899989,-2.235174e-08,285.83145,504.52332,67.996650,0.068534,28.0,1.0,2022.0


In [353]:
0.3*len(df)


751.8

In [354]:


s=df.head(1755)



f=df.tail(751)

In [355]:
s

,AOD_440nm,AE,windspeed,wind_direction,TP,Temp,PBLH,RH,AAI,day,month,year
626,0.468706,0.960,1.378377,105.671068,1.768591e-04,302.06506,537.62410,74.383620,0.686675,19.0,9.0,2002.0
633,0.465906,1.800,1.907279,166.984458,5.938299e-05,300.52576,555.50290,78.481834,0.602194,26.0,9.0,2002.0
636,0.469908,1.619,1.800257,189.353322,4.302710e-07,301.04758,492.15730,74.346910,0.220123,29.0,9.0,2002.0
639,0.318449,1.787,2.023659,164.650015,0.000000e+00,300.10574,437.02628,67.287895,-0.334267,2.0,10.0,2002.0
651,0.485983,1.800,2.013939,16.996497,2.147285e-04,299.66187,591.35986,73.569340,0.018496,14.0,10.0,2002.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5908,0.196232,1.486,3.263507,166.067284,-2.235174e-08,293.36923,680.98663,41.741860,-0.288313,6.0,3.0,2017.0
5910,0.575205,1.743,2.647420,344.031854,1.004236e-04,296.57220,685.75140,43.137253,-0.358852,8.0,3.0,2017.0
5911,0.675227,0.238,2.178139,43.138722,8.387864e-04,297.01315,739.64680,45.277287,-0.003122,9.0,3.0,2017.0
5913,0.462556,0.033,1.655908,122.318505,1.046726e-03,291.68826,586.42740,75.106600,-0.068478,11.0,3.0,2017.0


In [356]:
X_train_val=s[df.columns.values[1::]]
X_test=f[df.columns.values[1::]]
y_train_val=s[df.columns.values[0:1]]
y_test=f[df.columns.values[0:1]]


# In[34]:


X_train,X_val,y_train,y_val=train_test_split(X_train_val,y_train_val,train_size=0.8)


In [357]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)


In [358]:
model = Sequential()


model.add(Dense(256,  input_dim=X_train.shape[1],activation='relu'))
model.add(Dense(200, activation='relu'))

model.add(Dense(128,  activation='relu'))
model.add(Dense(64,  activation='relu'))
model.add(Dense(32,  activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='linear'))


In [359]:
model.compile(loss='mean_squared_error', optimizer='SGD')

In [360]:
model.fit(X_train, y_train, epochs=50, batch_size=32)

Epoch 1/50
44/44 [==============================] - 1s 5ms/step - loss: 0.1461
Epoch 2/50
44/44 [==============================] - 0s 8ms/step - loss: 0.0646
Epoch 3/50
44/44 [==============================] - 0s 7ms/step - loss: 0.0603
Epoch 4/50
44/44 [==============================] - 1s 18ms/step - loss: 0.0566
Epoch 5/50
44/44 [==============================] - 0s 8ms/step - loss: 0.0534
Epoch 6/50
44/44 [==============================] - 0s 10ms/step - loss: 0.0508
Epoch 7/50
44/44 [==============================] - 0s 6ms/step - loss: 0.0486
Epoch 8/50
44/44 [==============================] - 0s 8ms/step - loss: 0.0466
Epoch 9/50
44/44 [==============================] - 0s 6ms/step - loss: 0.0455
Epoch 10/50
44/44 [==============================] - 0s 6ms/step - loss: 0.0442
Epoch 11/50
44/44 [==============================] - 0s 6ms/step - loss: 0.0431
Epoch 12/50
44/44 [==============================] - 0s 7ms/step - loss: 0.0424
Epoch 13/50
44/44 [============================

In [361]:
print('Best: %f using %s' % (grid_result.best_score_, grid_result.best_params_))

Best: -0.043216 using {'activation': 'relu', 'num_layers': 2, 'num_neurons': 32}


In [362]:
y_test_pred = model.predict(X_test)

24/24 [==============================] - 0s 5ms/step


In [363]:
print("For testing Data")
#y_test_pred=model_rs.predict(X_test)
error_rs3=mean_absolute_error(y_test,y_test_pred)
print("MAE : % f" %(error_rs3))
rmse_rs3 = np.sqrt(MSE(10**y_test, 10**y_test_pred))
print("RMSE : % f" %(rmse_rs3))
r_sqr_rs3=r2_score(10**y_test, 10**y_test_pred)
print("R_squared : %f" %(r_sqr_rs3))


For testing Data
MAE :  0.168653
RMSE :  31.147715
R_squared : -27.710245


In [320]:
print("For testing Data")
#y_test_pred=model_rs.predict(X_test)
error_rs3=mean_absolute_error(y_test,y_test_pred)
print("MAE : % f" %(error_rs3))
rmse_rs3 = np.sqrt(MSE(y_test, y_test_pred))
print("RMSE : % f" %(rmse_rs3))
r_sqr_rs3=r2_score(y_test, y_test_pred)
print("R_squared : %f" %(r_sqr_rs3))


For testing Data
MAE :  0.152248
RMSE :  0.202019
R_squared : 0.466066


In [305]:
print("For testing Data")
#y_test_pred=model_rs.predict(X_test)
error_rs3=mean_absolute_error(y_test,y_test_pred)
print("MAE : % f" %(error_rs3))
rmse_rs3 = np.sqrt(MSE(y_test, y_test_pred))
print("RMSE : % f" %(rmse_rs3))
r_sqr_rs3=r2_score(y_test, y_test_pred)
print("R_squared : %f" %(r_sqr_rs3))



For testing Data
MAE :  0.160158
RMSE :  0.209877
R_squared : 0.423723
